In [2]:
import pandas as pd
import re

In [231]:
!gdown --id 15_FfenvHFoJVcPu6gGx9osLn71uDJ-Et

Downloading...
From: https://drive.google.com/uc?id=15_FfenvHFoJVcPu6gGx9osLn71uDJ-Et
To: /content/vacancies.csv
100% 1.05G/1.05G [00:06<00:00, 153MB/s]


In [215]:
df = pd.read_csv("vacancies.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (18,22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Парсер навыков из вакансий с [гитхаба](https://github.com/keshavbansal015/Job-Description-Parser)

P.S. другие парсеры (в основном платные и для CV):


1.   https://www.maprecruit.ai/parsers - 30 файлов потестила в демо
2.   https://docs.microsoft.com/en-us/connectors/candidatezip/#parse-jd-via-file-content - не смогла зарегистирироваться
3.   http://www.reshufl.com/demo/ - потестила демо



In [229]:
!gdown --id 179NC9uvqIbyahO1R3yyiYHV2srB38-5-

Downloading...
From: https://drive.google.com/uc?id=179NC9uvqIbyahO1R3yyiYHV2srB38-5-
To: /content/Resume_JD parser(spaCy).zip
100% 205M/205M [00:01<00:00, 113MB/s]


In [ ]:
import zipfile
zf = zipfile.ZipFile("parser.zip")
zf.extractall()

In [ ]:
# !pip install spacy
import spacy
sys.path.append(os.path.abspath('./Resume_JD parser(spaCy)/src'))

from func import *
from constants import *
from init_parser import init_parser_func
from tqdm import tqdm
sys.path.append(os.path.abspath('..'))

nlp = spacy.load('en_core_web_sm')

if __name__ == '__main__':)
	matcher = init_parser_func(nlp, './Resume_JD parser(spaCy)/input/Skillset.xlsx', file_type="excel")
	
	final_database = pd.DataFrame()	

	for each in tqdm(range(len(df))):

		text = df.loc[each, 'description']
		application_subject = df.loc[each,'id']
		data = create_profile(nlp,matcher,text,application_subject)
		final_database = final_database.append(data)

	# Saving the database
	final_database.to_csv('./Resume_JD parser(spaCy)/input/Data.csv', index=False)

/content
Success!


100%|██████████| 164916/164916 [2:08:26<00:00, 21.40it/s]


In [ ]:
final_database

,Company/Candidate Name,Skill,Sub-skill,Count
0,47319343,Coding,SQL,1
0,47472541,Analytics,Excel,1
0,46134407,Business,MS Office,1
0,47513275,Business,MS Office,1
0,47727709,Coding,JavaScript,1
...,...,...,...,...
0,46120872,Analytics,Excel,1
0,47121032,Business,MS Office,1
0,47187542,DevOps,MS Word,1
1,47187542,Analytics,Excel,1


# Парсер описания вакансий с помощью регулярных выражений

In [252]:
def JD(x: str, regular_try: str, regular_except: str, regular_except_try: str, result: list):
  """
  Создаём метод, который ищет по заданному регулярному выражению в строке
  Аргументы:
  x - строка в датафрейме с описанием вакансий
  regular_try, regular_except, regular_except_try - регулярные выражения, согласно которым работает парсер
  result - список, куда записывается результат работы метода
  """
  try:
    result.append(re.search(regular_try, x).group(0))
  except:
    try:
      result.append(re.search(regular_except, x).group(0))
    except:
      try:
        result.append(re.search(regular_except_try, x).group(0))
      except:
        result.append(re.search(regular_except, x))

In [250]:
def JD_new_column(regular_try: str, regular_except: str, regular_except_try: str, new_column: str, df: pd.DataFrame):
  """
  Создаём метод, который создаёт новый столбец после парсинга
  Аргументы:
  regular_try, regular_except, regular_except_try - регулярные выражения, согласно которым работает парсер
  new_column - название столбца, который создаётся
  df - датафрейм, который содержит описание вакансий
  """
  result = []
  df['description'].apply(lambda x: JD(x, regular_try, regular_except, regular_except_try, result));
  df[f'description_{new_column}'] = pd.DataFrame(result)[0]


In [260]:
JD_new_column(r'(?i)обязанности:.*?(?=(<strong>|<\/ul>))|чем предстоит заниматься.*?(?=(<strong>|<\/ul>))|(задачи|задачах):.*?(?=(<strong>|<\/ul>))|ищем.{1,25}которые\s{1,}будут.*?(?=(<strong>|<\/ul>))|что нужно делать.*?(?=(<strong>|<\/ul>))', 
              r'(?i)обязанности.*?\.|чем предстоит заниматься.*?\.|(задачи|задачах).*?\.|ищем.{1,15}которые\s{1,}будут.*?\.|что нужно делать.*?\.',
              r'(?i)обязанности.+|чем предстоит заниматься.+|(задачи|задачах).+|ищем.{1,15}которые\s{1,}будут.+|что нужно делать.+',
                       'responsibilities', df);

In [254]:
JD_new_column(r'(?i)(требования|требование):.*?(?=(<strong>|<\/ul>))', 
              r'(?i)(требования|требование).*?\.',
              r'(?i)(требования|требование).+',
                       'requirements', df);

In [255]:
JD_new_column(r'(?i)условия:.*?(?=(<strong>|<\/ul>))|предлагаем.*?(?=(<strong>|<\/ul>))', 
              r'(?i)условия.*?\.|предлагаем.*?\.',
              r'(?i)условия.+|предлагаем.+',
                       'conditions', df);

In [257]:
JD_new_column(r'(?i)о компании:.*?(?=(<strong>|<\/ul>))|о нас:.*?(?=(<strong>|<\/ul>))', 
              r'(?i)о компании:.*?\.|о нас:.*?\.',
              r'(?i)о компании:.+|о нас:.+',
                       'about', df);

In [262]:
df[['description_responsibilities', 'description_requirements', 'description_conditions', 'description_about']].head()

,description_responsibilities,description_requirements,description_conditions,description_about
0,ОБЯЗАННОСТИ:</strong></p> <ul> <li>Организация...,Требование:</strong></p> <ul> <li>Опыт работы ...,ПРЕДЛАГАЕМ:</strong></p> <ul> <li>Официальное ...,О НАС:</strong></p> <ul> <li>высокий уровень а...
1,Что нужно делать</strong><br />Доставлять банк...,None,предлагаем</strong></p> <ul> <li>2/2 или 5/2. ...,None
2,Обязанности:</strong></p> <ul> <li>Сварка кузо...,Требования:</strong> <ul> <li>Опыт работы от 3...,Условия:</strong> <ul> <li>Трудоустройство сог...,None
3,Обязанности:<br />- Контроль и координация раб...,Требования:<br />- Опыт работы в продажах прив...,"Условия:<br />- Оплата без задержек, премии, б...",None
4,Обязанности:</strong> <ul> <li>ведение первичн...,Требования:</strong> <ul> <li>высшее образован...,Условия:</strong> <ul> <li>5 дневный рабочий д...,None


In [263]:
df[['description_responsibilities', 'description_requirements', 'description_conditions', 'description_about']].describe()

,description_responsibilities,description_requirements,description_conditions,description_about
count,138417,124890,143443,1206
unique,104987,94967,103509,782
top,Обязанности:</strong></p><ul><li>Обслуживание ...,Требования: </strong></p><ul><li>Ответственнос...,Условия:</strong></p><ul><li>Оформление в соот...,О нас:</strong></p> <p>
freq,1264,1263,1093,70


In [264]:
df[['description', 'description_responsibilities', 'description_requirements', 'description_conditions', 'description_about']].to_csv('dataframe.csv', index=False)